<a href="https://colab.research.google.com/github/IANGECHUKI176/deeplearning/blob/main/pytorch/nlp/karpathy/gpt_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-08-19 13:49:04--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2023-08-19 13:49:04 (16.8 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
with open('input.txt',"r") as f:
    text = f.read()



In [ ]:
print('length of dataset in characters',len(text))

length of dataset in characters 1115394


In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
#mapping from characters to integers
itos = {i:ch for i,ch in enumerate(chars)}
stoi = {ch:i for i,ch in itos.items()}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

encoded  = encode('Hi There')
decoded = decode(encoded)
print(encoded)
print(decoded)

[20, 47, 1, 32, 46, 43, 56, 43]
Hi There


In [ ]:
person = {
    "name":'ian',
    "age": 34
}
person.items()

dict_items([('name', 'ian'), ('age', 34)])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt


In [ ]:
data = torch.tensor(encode(text),dtype = torch.long)
data.shape , data.dtype
data[:500]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [ ]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x = train_data[:block_size ]
y= train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context} the target is {target}')


When input is tensor([18]) the target is 47
When input is tensor([18, 47]) the target is 56
When input is tensor([18, 47, 56]) the target is 57
When input is tensor([18, 47, 56, 57]) the target is 58
When input is tensor([18, 47, 56, 57, 58]) the target is 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [ ]:
torch.manual_seed(1332)
batch_size = 4
block_size = 8 #max seq length for prediction
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size ] for i in ix])
    y = torch.stack([data[i+1:i+block_size +1] for i in ix])
    return x,y

xb,yb = get_batch("train")
print('input')
print(xb.shape)
print(xb)
print('target')
print(yb.shape)
print(yb)
print('-------------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]

        print(f'When the input is {context.tolist()} the target is {target}')

input
torch.Size([4, 8])
tensor([[44,  1, 61, 47, 58, 46,  1, 63],
        [27, 44,  1, 53, 59, 56,  1, 45],
        [40, 59, 58,  1, 39,  1, 57, 50],
        [ 1, 61, 46, 39, 58,  6,  1, 53]])
target
torch.Size([4, 8])
tensor([[ 1, 61, 47, 58, 46,  1, 63, 53],
        [44,  1, 53, 59, 56,  1, 45, 56],
        [59, 58,  1, 39,  1, 57, 50, 47],
        [61, 46, 39, 58,  6,  1, 53,  5]])
-------------
When the input is [44] the target is 1
When the input is [44, 1] the target is 61
When the input is [44, 1, 61] the target is 47
When the input is [44, 1, 61, 47] the target is 58
When the input is [44, 1, 61, 47, 58] the target is 46
When the input is [44, 1, 61, 47, 58, 46] the target is 1
When the input is [44, 1, 61, 47, 58, 46, 1] the target is 63
When the input is [44, 1, 61, 47, 58, 46, 1, 63] the target is 53
When the input is [27] the target is 44
When the input is [27, 44] the target is 1
When the input is [27, 44, 1] the target is 53
When the input is [27, 44, 1, 53] the target i

In [ ]:
n_embd = 32
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super(BigramLanguageModel,self).__init__()
        #each token directly reads off the logits for the next tken from lookup table
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.position_embedding_table = nn.Embedding(block_size,n_embd)
        self.lm_head = nn.Linear(n_embd,vocab_size)
    def forward(self, idx, targets=None):
        B,T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        token_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T)) # (T,C)
        x = token_emb + pos_emb
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    def generate(self,idx,max_new_tokens):
        #idx is (B,T) array of indices in the currect context
        for _ in range(max_new_tokens):
            logits,loss = self(idx)
            #focus on last time step
            logits = logits[:,-1,:] #becomes (B,C)
            #apply softmax to get probs
            probs = F.softmax(logits,dim = -1) # (B,C)
            #sample from distribution
            idx_next = torch.multinomial(probs,num_samples = 1) #(B,1)
            #append sampled index to the running sequence
            idx = torch.cat((idx, idx_next),dim = 1) #(B,T+ 1)
        return idx

In [ ]:
m = BigramLanguageModel()
logits,loss = m(xb,yb)
# print(logits.shape)
# print(loss)

In [ ]:
#print(decode(m.generate(idx = torch.zeros((1,1),dtype = torch.long) ,max_new_tokens=100)[0].tolist()))
context = torch.zeros((1, 1), dtype=torch.long, device='cpu')
print(decode(m.generate(context, max_new_tokens=5)[0].tolist()))


Pc,pH


In [ ]:
optimizer = torch.optim.Adam(m.parameters(),lr = 1e-1)

In [ ]:
batch_size = 32
for steps in range(100):
    xb , yb = get_batch('train')

    logits ,loss = m(xb,yb)
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    if steps % 100 == 0:
        print(loss.item())

4.429553985595703


In [ ]:
print(decode(m.generate(idx = torch.zeros((1,1),dtype = torch.long) ,max_new_tokens=5)[0].tolist()))


Makis


In [ ]:
max_iters = 500
eval_iters = 200
eval_interval = 500
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long)
#print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 2.6714, val loss 2.7034
step 499: train loss 2.7150, val loss 2.7429


The mathematical trick in self attention

In [ ]:
torch.manual_seed(1337)
B , T, C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
torch.tril(torch.ones(4,4))

tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])

In [ ]:
#version 1
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #(T,C)
        xbow[b,t] = torch.mean(xprev,0)

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a/torch.sum(a,1,keepdim = True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print('a=')
print(a)
print('-------')
print('b=')
print(b)
print('-------')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
-------
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
-------
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
#version 2
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1,keepdim = True)
xbow2 = wei @ x # (T,T) @ (B,T,C) ---> (B,T,C)
xbow2.shape

torch.Size([4, 8, 2])

In [ ]:
#version 3 use softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0,float('-inf'))
wei = wei.softmax(1)
xbow3 = wei @ x
xbow3.shape

torch.Size([4, 8, 2])

In [ ]:
#version 4
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C) #batch, time,channels

#lets see a single head perfom self attention
head_size = 16
key = nn.Linear(C,head_size,bias = False)
query = nn.Linear(C,head_size,bias = False)
value = nn.Linear(C,head_size,bias = False)
k = key(x) # (B,T,16)
q = query(x) # (B,T,16)
wei = q @ k.transpose(-2,-1) # (B,T,16) @ (B,16,T )----> (B,T,T)

tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0,float('-inf'))
wei = F.softmax(wei,dim = 1)
v = value(x)
out = wei @ v  # (B,T,16)

# out = wei @ x
out.shape

torch.Size([4, 8, 16])

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1) * head_size ** -0.5

In [ ]:
k.var(),q.var(),wei.var()

(tensor(1.0449), tensor(1.0700), tensor(1.0918))

Hyperparams

In [ ]:
n_layers = 6
dropout = 0.2
n_head = 6
n_embd = 384
block_size = 256
eval_iters = 200
eval_interval = 500
max_iters = 5000
lr = 3e-4

In [ ]:
class Head(nn.Module):
    def __init__(self,head_size):
        super(Head,self).__init__()

        self.key = nn.Linear(n_embd,head_size,bias = False)
        self.query = nn.Linear(n_embd,head_size,bias = False)
        self.value = nn.Linear(n_embd,head_size,bias = False)
        self.register_buffer("tril",torch.tril(torch.ones(block_size,block_size)))
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):
        B,T,C= x.shape
        k = self.key(x) #(B,T,C)
        q= self.query(x) #(B,T,C)
        #compute attention scores ('affinities)
        wei = q @ k.transpose(-2,-1) * C** - 0.5 # (B,T,C) @ (B,C,T) ---> (B,T,T)
        wei = wei.masked_fill(self.tril[:T,:T] == 0,float('-inf')) # (B,T,T)
        wei = F.softmax(wei,dim= -1) # (B,T,T)
        wei = self.dropout(wei)
        #perfom weighted aggregation of the values
        x = self.value(x)
        out = wei @ x
        return out #(B,T,T) @ (B,T,hs) ---> (B,T,hs)

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self,num_heads,head_size):
        super(MultiHeadAttention,self).__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd,n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads],dim = -1)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
class FeedForward(nn.Module):
    def __init__(self,n_embd):
        super(FeedForward,self).__init__()

        self.net = nn.Sequential(
            nn.Linear(n_embd,n_embd * 4),
            nn.ReLU(),
            nn.Linear(n_embd * 4,n_embd),
            nn.Dropout(dropout)
        )
    def forward(self,x):
        return self.net(x)

In [ ]:
class Block(nn.Module):
    def __init__(self,n_embd,n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head,head_size) # i.e 4 heads of 8 dim self-attention
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self,x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:

class LanguageModel(nn.Module):
    def __init__(self):
        super(LanguageModel,self).__init__()
        #each token directly reads off the logits for the next tken from lookup table
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.position_embedding_table = nn.Embedding(block_size,n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd,n_head = n_head) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd,vocab_size)
    def forward(self, idx, targets=None):
        B,T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        token_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T)) # (T,C)
        x = token_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) #apply one head of self-attention (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    def generate(self,idx,max_new_tokens):
        #idx is (B,T) array of indices in the currect context
        for _ in range(max_new_tokens):
               # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            #focus on last time step
            logits = logits[:,-1,:] #becomes (B,C)
            #apply softmax to get probs
            probs = F.softmax(logits,dim = -1) # (B,C)
            #sample from distribution
            idx_next = torch.multinomial(probs,num_samples = 1) #(B,1)
            #append sampled index to the running sequence
            idx = torch.cat((idx, idx_next),dim = 1) #(B,T+ 1)
        return idx

In [ ]:
m = LanguageModel()
optimizer = torch.optim.Adam(m.parameters(),lr = lr )
max_iters = 5000


In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

KeyboardInterrupt: ignored

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))